## Dropped Index: 
- Alpha30(要用到fama三因子)
- Alpha75(要用到BENCHMARKINDEX)
- Alpha143(要用到SELF函数)
- Alpha149(要用到BENCHMARKINDEX)
- Alpha181(要用到BENCHMARKINDEX)
- Alpha182(要用到BENCHMARKINDEX)

In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, math
%matplotlib inline

In [2]:
BTC_data = pd.read_pickle('BTC_data.pickle')

In [3]:
BTC_data.columns

Index(['price_close', 'price_high', 'price_low', 'price_open',
       'volume_traded'],
      dtype='object')

In [4]:
from BTC_Alpha_func import *

### **先罗列具体alpha因子的计算之前，先按照国泰研报中的表15的顺序写好常用的函数**

In [5]:
OPEN = BTC_data['price_open'].fillna(method='ffill')
HIGH = BTC_data['price_high']
LOW = BTC_data['price_low']
CLOSE = BTC_data['price_close']
VOLUME = BTC_data['volume_traded']
VWAP = CLOSE
RET = RET(CLOSE)
AMOUNT = VOLUME * CLOSE
DTM = (OPEN.values <= DELAY(OPEN, 1).values) \
            * MAX(HIGH - OPEN, OPEN - DELAY(OPEN, 1))
DBM = (OPEN >= DELAY(OPEN, 1).values) \
            * MAX(OPEN - LOW, OPEN - DELAY(OPEN, 1))
TR = MAX(MAX(HIGH-LOW,ABS(HIGH-DELAY(CLOSE,1))),ABS(LOW-DELAY(CLOSE,1)) )
HD = HIGH-DELAY(HIGH,1)
LD = DELAY(LOW,1)-LOW

#OPEN.isnull().values.any()

In [6]:
'''
This part was designed to test if there is NaN value in these 
raw data, the ideal result is all 'False'
'''

variable_name = [OPEN, HIGH, LOW, CLOSE, VOLUME,\
                 VWAP, RET, AMOUNT, DTM, DBM, TR, HD, LD]
index = 0
for item in variable_name:
    index += 1
    print ('Does No.%s variable have NaN values? %s\n'%(str(index), str(item.isnull().values.any())))


Does No.1 variable have NaN values? False

Does No.2 variable have NaN values? False

Does No.3 variable have NaN values? False

Does No.4 variable have NaN values? False

Does No.5 variable have NaN values? False

Does No.6 variable have NaN values? False

Does No.7 variable have NaN values? False

Does No.8 variable have NaN values? False

Does No.9 variable have NaN values? False

Does No.10 variable have NaN values? False

Does No.11 variable have NaN values? False

Does No.12 variable have NaN values? False

Does No.13 variable have NaN values? False



In [7]:
Alpha1 = -1 * CORR(RANK(DELTA(LOG(VOLUME),1)), RANK((CLOSE-OPEN)/OPEN), 6)

In [8]:
##会出现NAN，因为HIGH = LOW, 出现NaN的地方设置为0，因为此时分子分母都为零
Alpha2 = (-1 * DELTA((((CLOSE - LOW) - (HIGH - CLOSE)) / (HIGH - LOW)), 1)).fillna(0)

### 对于：？较为复杂的表达式，我都先用一些中间变量存储中间运算的结果，以下均采用这一做法，不赘述

In [9]:
cache = CLOSE - ((~(CLOSE>DELAY(CLOSE,1)))*MAX(HIGH,DELAY(CLOSE,1)) + (~(CLOSE>DELAY(CLOSE,1)))*MAX(HIGH,DELAY(CLOSE,1)))
Alpha3 = SUM((~(CLOSE==DELAY(CLOSE,1)) * cache), 6)

In [10]:
#tail计算的是倒数第二个冒号后面的结果
tail = (((1 < (VOLUME / MEAN(VOLUME,20))) | 
  ((VOLUME / MEAN(VOLUME,20)) == 1)) * 1  - ~((VOLUME / MEAN(VOLUME,20)) == 1) * (-1))

#med计算的是中间的一个判断句（第1个冒号之后）的结果
med = ((SUM(CLOSE, 2) / 2) < ((SUM(CLOSE, 8) / 8) - STD(CLOSE, 8))) * 1 + \
        ~(((SUM(CLOSE, 2) / 2) < ((SUM(CLOSE, 8) / 8) - STD(CLOSE, 8)))) * tail

Alpha4 = (((SUM(CLOSE, 8) / 8) + STD(CLOSE, 8)) < (SUM(CLOSE, 2) / 2)) * (-1) + ~(((SUM(CLOSE, 8) / 8) + STD(CLOSE, 8)) < (SUM(CLOSE, 2) / 2)) * med

In [11]:
Alpha5 = (-1 * TSMAX(CORR(TSRANK(VOLUME, 5), TSRANK(HIGH, 5), 5), 3))

In [12]:
Alpha6 = (RANK(SIGN(DELTA((((OPEN * 0.85) + (HIGH * 0.15))), 4)))* (-1))

In [13]:
Alpha7 = ((RANK(MAX((VWAP - CLOSE), 3)) + RANK(MIN((VWAP - CLOSE), 3))) * RANK(DELTA(VOLUME, 3)))

In [14]:
Alpha8 = RANK(DELTA(((((HIGH + LOW) / 2) * 0.2) + (VWAP * 0.8)), 4) * -1)

In [15]:
Alpha8 = RANK(DELTA(((((HIGH + LOW) / 2) * 0.2) + (VWAP * 0.8)), 4) * -1)

In [16]:
Alpha9 = SMA(((HIGH+LOW)/2-(DELAY(HIGH,1)+DELAY(LOW,1))/2)*(HIGH-LOW)/VOLUME,7,2)

In [17]:
Alpha10 = RANK(MAX((STD(RET, 20) * (RET < 0) + CLOSE**2 * ~(RET < 0) * CLOSE)**2, 5))

In [18]:
Alpha11 = SUM(((CLOSE-LOW)-(HIGH-CLOSE))/(HIGH-LOW)*VOLUME,6)

In [19]:
Alpha12 = (RANK((OPEN - (SUM(VWAP, 10) / 10)))) * (-1 * (RANK(ABS((CLOSE - VWAP)))))

In [20]:
Alpha13 = (((HIGH * LOW)**0.5) - VWAP)

In [21]:
Alpha14 = CLOSE-DELAY(CLOSE,5)

In [22]:
Alpha15 = OPEN/DELAY(CLOSE,1)-1

In [23]:
Alpha16 = (-1 * TSMAX(RANK(CORR(RANK(VOLUME), RANK(VWAP), 5)), 5))

In [24]:
Alpha17 = RANK((VWAP - MAX(VWAP, 15)))**DELTA(CLOSE, 5)

In [25]:
Alpha18 = CLOSE/DELAY(CLOSE,5)

In [26]:
Alpha19 = (CLOSE-DELAY(CLOSE,5))/DELAY(CLOSE,5) * (CLOSE<DELAY(CLOSE,5)) + \
        ~(CLOSE<DELAY(CLOSE,5)) * (0*(CLOSE==DELAY(CLOSE,5))+ ~(CLOSE==DELAY(CLOSE,5))* (CLOSE-DELAY(CLOSE,5))/CLOSE)

In [27]:
Alpha20 = (CLOSE-DELAY(CLOSE,6))/DELAY(CLOSE,6)*100

In [28]:
Alpha21 = REGBETA(MEAN(CLOSE,6),SEQUENCE(6),6)

In [29]:
'''SMEAN(**arg, arg1, arg2) is not defined, we must figure out its definition first'''
Alpha22 = MEAN((CLOSE-MEAN(CLOSE,6))/MEAN(CLOSE,6)-DELAY((CLOSE-MEAN(CLOSE,6))/MEAN(CLOSE,6),3),12)

In [30]:
Alpha23 = SMA((CLOSE>DELAY(CLOSE,1))*STD(CLOSE,20),20,1)/\
(SMA((CLOSE>DELAY(CLOSE,1))*STD(CLOSE,20),20,1 )+SMA((CLOSE<=DELAY(CLOSE,1))*STD(CLOSE,20),20,1))*100

In [31]:
Alpha24 = SMA(CLOSE-DELAY(CLOSE,5),5,1)

In [32]:
Alpha25 = ((-1 * RANK((DELTA(CLOSE, 7) * (1 - RANK(DECAYLINEAR((VOLUME / MEAN(VOLUME,20)), 9)))))) * (1 + RANK(SUM(RET, 250))))

In [33]:
Alpha26 = ((((SUM(CLOSE, 7) / 7) - CLOSE)) + ((CORR(VWAP, DELAY(CLOSE, 5), 230))))

In [34]:
Alpha27 = WMA((CLOSE-DELAY(CLOSE,3))/DELAY(CLOSE,3)*100+(CLOSE-DELAY(CLOSE,6))/DELAY(CLOSE,6)*100,12)

In [35]:
Alpha28 = 3*SMA((CLOSE-TSMIN(LOW,9))/(TSMAX(HIGH,9)-TSMIN(LOW,9))*100,3,1)-2*SMA(SMA((CLOSE-TSMIN(LOW,9))/( MAX(HIGH,9)-TSMAX(LOW,9))*100,3,1),3,1)

In [36]:
Alpha29 = (CLOSE-DELAY(CLOSE,6))/DELAY(CLOSE,6)*VOLUME

In [37]:
#Alpha30 drop, 因为这里要用到MKT，SMB，HML， thus drop it

In [38]:
Alpha31 = (CLOSE-MEAN(CLOSE,12))/MEAN(CLOSE,12)*100

In [39]:
Alpha32 = (-1 * SUM(RANK(CORR(RANK(HIGH), RANK(VOLUME), 3)), 3))

In [40]:
Alpha33 = ((((-1 * TSMIN(LOW, 5)) + DELAY(TSMIN(LOW, 5), 5)) * RANK(((SUM(RET, 240) - SUM(RET, 20)) / 220))) * TSRANK(VOLUME, 5))

In [41]:
Alpha34 = MEAN(CLOSE,12)/CLOSE

In [42]:
Alpha35 = (MIN(RANK(DECAYLINEAR(DELTA(OPEN, 1), 15)), RANK(DECAYLINEAR(CORR((VOLUME), ((OPEN * 0.65) + (OPEN *0.35)), 17),7))) * -1)

In [43]:
Alpha36 = RANK(SUM(CORR(RANK(VOLUME), RANK(VWAP), 6), 2))

In [44]:
Alpha37 = (-1 * RANK(((SUM(OPEN, 5) * SUM(RET, 5)) - DELAY((SUM(OPEN, 5) * SUM(RET, 5)), 10))))

In [45]:
Alpha38 = ((SUM(HIGH, 20) / 20) < HIGH) * (-1 * DELTA(HIGH, 2)) + 0 * ~(SUM(HIGH, 20) / 20 < HIGH)

In [46]:
Alpha39 = ((RANK(DECAYLINEAR(DELTA((CLOSE), 2),8)) - RANK(DECAYLINEAR(CORR(((VWAP * 0.3) + (OPEN * 0.7)), SUM(MEAN(VOLUME,180), 37), 14), 12))) * -1)

In [47]:
Alpha40 = SUM((CLOSE>DELAY(CLOSE,1))*VOLUME+ 0*~(CLOSE>DELAY(CLOSE,1)),26)\
               /SUM(CLOSE<=DELAY(CLOSE,1)*VOLUME+0*~(CLOSE<=DELAY(CLOSE,1)), 26)*100

In [48]:
Alpha41 = (RANK(MAX(DELTA((VWAP), 3), 5))* -1)

In [49]:
Alpha42 = ((-1 * RANK(STD(HIGH, 10))) * CORR(HIGH, VOLUME, 10))

In [50]:
Alpha43 = SUM(VOLUME * (CLOSE>DELAY(CLOSE,1)) + (~(CLOSE>DELAY(CLOSE,1))) * (-VOLUME) * (CLOSE<DELAY(CLOSE,1)),6)

In [51]:
Alpha44 = (TSRANK(DECAYLINEAR(CORR(((LOW)), MEAN(VOLUME,10), 7), 6),4) + TSRANK(DECAYLINEAR(DELTA((VWAP), 3), 10), 15))

In [52]:
Alpha45 = (RANK(DELTA((((CLOSE * 0.6) + (OPEN *0.4))), 1)) * RANK(CORR(VWAP, MEAN(VOLUME,150), 15)))

In [53]:
Alpha46 = (MEAN(CLOSE,3)+MEAN(CLOSE,6)+MEAN(CLOSE,12)+MEAN(CLOSE,24))/(4*CLOSE)

In [54]:
Alpha47 = SMA((TSMAX(HIGH,6)-CLOSE)/(TSMAX(HIGH,6)-TSMIN(LOW,6))*100,9,1)

In [55]:
Alpha48 = (-1*((RANK(((SIGN((CLOSE - DELAY(CLOSE, 1))) + SIGN((DELAY(CLOSE, 1) - DELAY(CLOSE, 2)))) + SIGN((DELAY(CLOSE, 2) - DELAY(CLOSE, 3)))))) * SUM(VOLUME, 5)) / SUM(VOLUME, 20))

In [56]:
'''
SUM(((HIGH+LOW)>=(DELAY(HIGH,1)+DELAY(LOW,1))?0:

MAX(ABS(HIGH-DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1)))),12)/
    (SUM(((HIGH+LOW)>=(DELAY(HIGH,1)+DELAY(LOW,1))?0:
    
MAX(ABS(HIGH-DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1)))),12)
    +SUM(((HIGH+LOW)<=(DELAY(HIGH,1)+DELAY(LOW,1))?0:
    
MAX(ABS(HIGH - DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1)))),12))
'''


tail = SUM(
                (
                    ((HIGH+LOW)<=(DELAY(HIGH,1)+DELAY(LOW,1))) * 0 + 
                    ~((HIGH+LOW)<=(DELAY(HIGH,1)+DELAY(LOW,1)))
                    *MAX(ABS(HIGH -DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1)))
                )
            ,12)


dividend = SUM(
                    ((HIGH+LOW)>=(DELAY(HIGH,1)+DELAY(LOW,1))) * 0 +
                    ~((HIGH+LOW)>=(DELAY(HIGH,1)+DELAY(LOW,1))) * 
                        MAX(ABS(HIGH-DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1)))
                ,12) + cache



divisor = SUM(
                ((HIGH+LOW)>=(DELAY(HIGH,1)+DELAY(LOW,1))) * 0
                + ~((HIGH+LOW)>=(DELAY(HIGH,1)+DELAY(LOW,1))) *
                    MAX(ABS(HIGH-DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1)))
                ,12)

Alpha49 = divisor/dividend

In [57]:
'''
SUM(((HIGH+LOW)<=(DELAY(HIGH,1)+DELAY(LOW,1))?0:

MAX(ABS(HIGH-DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1)))),12)
    /(SUM(((HIGH+LOW)<=(DELAY(HIGH,1)+DELAY(LOW,1))?0:
    
MAX(ABS(HIGH -DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1)))),12)
    +SUM(((HIGH+LOW)>=(DELAY(HIGH,1)+DELAY(LOW,1))?0:
    
MAX(ABS(HIGH -DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1)))),12))
    
----- 到这里 前面这部分的结构是A/B，与上一个index计算方法基本一致 -----   
    
-

SUM(((HIGH+LOW)>=(DELAY(HIGH,1)+DELAY(LOW,1))?0:
    
MAX(ABS(HIGH-DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1)))),12)/
    (SUM(((HIGH+LOW)>=(DELAY(HIGH,1)+DELAY(LOW,1))?0: 
    
MAX(ABS(HIGH-DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1)))),12)
    +SUM(((HIGH+LOW)<=(DELAY(HIGH,1)+DELAY(LOW,1))?0:

MAX(ABS(HIGH-DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1)))),12))

-----减去的这部分就是前一个index，解析式一模一样
'''
tail = SUM(
                (
                    ((HIGH+LOW)>=(DELAY(HIGH,1)+DELAY(LOW,1))) * 0 + 
                    ~((HIGH+LOW)>=(DELAY(HIGH,1)+DELAY(LOW,1)))
                    *MAX(ABS(HIGH -DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1)))
                )
            ,12)


dividend = SUM(
                    ((HIGH+LOW)<=(DELAY(HIGH,1)+DELAY(LOW,1))) * 0 +
                    ~((HIGH+LOW)<=(DELAY(HIGH,1)+DELAY(LOW,1))) * 
                        MAX(ABS(HIGH-DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1)))
                ,12) + cache



divisor = SUM(
                ((HIGH+LOW)<=(DELAY(HIGH,1)+DELAY(LOW,1))) * 0
                + ~((HIGH+LOW)<=(DELAY(HIGH,1)+DELAY(LOW,1))) *
                    MAX(ABS(HIGH-DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1)))
                ,12)

#Alpha50跟Alpha49不独立唉。。不知道为什么要这样设置

Alpha50 = divisor/dividend - Alpha49

In [58]:
'''Alpha51
SUM(((HIGH+LOW)<=(DELAY(HIGH,1)+DELAY(LOW,1))?0:

MAX(ABS(HIGH-DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1)))),12)/
    (SUM(((HIGH+LOW)<=(DELAY(HIGH,1)+DELAY(LOW,1))?0:

MAX(ABS(HIGH -DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1)))),12)
    +SUM(((HIGH+LOW)>=(DELAY(HIGH,1)+DELAY(LOW,1))?0:

MAX(ABS(HIGH -DELAY(HI GH,1)),ABS(LOW-DELAY(LOW,1)))),12))
'''
Alpha51 = Alpha50 + Alpha49

In [59]:
##把表达式中的L替换为LOW
Alpha52 = SUM(MAX(0,HIGH-DELAY((HIGH+LOW+CLOSE)/3,1)),26)/SUM(MAX(0,DELAY((HIGH+LOW+CLOSE)/3,1) - LOW),26)* 100

In [60]:
Alpha53 = COUNT(CLOSE>DELAY(CLOSE,1),12)/12*100

In [61]:
#STD() missing 1 required positional argument: 'n'，姑且设置为’10‘, 与后方一致
Alpha54 = (-1 * RANK((STD(ABS(CLOSE - OPEN), 10) + (CLOSE - OPEN)) + CORR(CLOSE, OPEN, 10)))

In [62]:
'''讲道理，直接想出这个解析式的真滴是大哥-。-
SUM(16*
(A)
    (CLOSE-DELAY(CLOSE,1)+(CLOSE-OPEN)/2+DELAY(CLOSE,1)-DELAY(OPEN,1))
    /
(B)
    ((ABS(HIGH-DELAY(CLOSE,1))>ABS(LOW-DELAY(CLOSE,1)) &
    ABS(HIGH-DELAY(CLOSE,1))>ABS(HIGH-DELAY(LOW,1)) ?   
    ABS(HIGH-DELAY(CLOSE,1))+ABS(LOW-DELAY(CLOSE,1))/2
    +ABS(DELAY(CLOSE,1)-DELAY(OPEN,1))/4 :   
    (ABS(LOW-DELAY(CLOSE,1))>ABS(HIGH-DELAY(LOW,1))    &
    ABS(LOW-DELAY(CLOSE,1))>ABS(HIGH-DELAY(CLOSE,1)) ?
    ABS(LOW-DELAY(CLOSE,1))+ABS(HIGH-DELAY(CLOSE,1))/2+
    ABS(DELAY(CLOSE,1)-DELAY(OPEN,1))/4:    
    ABS(HIGH-DELAY(LOW,1))+ABS(DELAY(CLOSE,1)-DELAY(OPEN,1))/4)))    
    *
(C)    
    MAX(ABS(HIGH-DELAY(CLOSE,1)),ABS(LOW-DELAY(CLOSE,1)))   
,20)

大致结构是SUM(A/B*C, n)，我真滴是很服气唉
'''
part_C = MAX(ABS(HIGH-DELAY(CLOSE,1)),ABS(LOW-DELAY(CLOSE,1)))

part_A = (CLOSE-DELAY(CLOSE,1)+(CLOSE-OPEN)/2+DELAY(CLOSE,1)-DELAY(OPEN,1))

part_B1_cond = (ABS(HIGH-DELAY(CLOSE,1))>ABS(LOW-DELAY(CLOSE,1))) & \
    (ABS(HIGH-DELAY(CLOSE,1))>ABS(HIGH-DELAY(LOW,1)))
    
part_B2_cond = (ABS(LOW-DELAY(CLOSE,1))>ABS(HIGH-DELAY(LOW,1))) & \
    (ABS(LOW-DELAY(CLOSE,1))>ABS(HIGH-DELAY(CLOSE,1)))

part_B = part_B1_cond *(ABS(HIGH-DELAY(CLOSE,1))+ABS(LOW-DELAY(CLOSE,1))/2 + ABS(DELAY(CLOSE,1)-DELAY(OPEN,1))/4)\
            + (~part_B1_cond)\
            * (part_B2_cond * (ABS(LOW-DELAY(CLOSE,1))+ABS(HIGH-DELAY(CLOSE,1))/2 + ABS(DELAY(CLOSE,1)-DELAY(OPEN,1))/4)\
            + (~part_B2_cond) * (ABS(HIGH-DELAY(LOW,1))+ABS(DELAY(CLOSE,1)-DELAY(OPEN,1))/4))

Alpha55 = SUM(part_A/part_B*part_C, 20)

In [63]:
Alpha56 = (RANK((OPEN - TSMIN(OPEN, 12))) < RANK((RANK(CORR(SUM(((HIGH + LOW)/2), 19), SUM(MEAN(VOLUME,40), 19), 13))**5)))

In [64]:
Alpha57 = SMA((CLOSE-TSMIN(LOW,9))/(TSMAX(HIGH,9)-TSMIN(LOW,9))*100,3,1)

In [65]:
Alpha58 = COUNT(CLOSE>DELAY(CLOSE,1),20)/20*100

In [66]:
Alpha59 = SUM((CLOSE==DELAY(CLOSE,1))*0+(CLOSE!=DELAY(CLOSE,1))*CLOSE \
        - ((CLOSE>DELAY(CLOSE,1))*MIN(LOW,DELAY(CLOSE,1)) + ~(CLOSE>DELAY(CLOSE,1)*MAX(HIGH,DELAY(CLOSE,1)))), 20)

In [67]:
Alpha60 = SUM(((CLOSE-LOW)-(HIGH-CLOSE))/(HIGH-LOW)*VOLUME,20)

In [68]:
Alpha61 = (MAX(RANK(DECAYLINEAR(DELTA(VWAP, 1),12)),RANK(DECAYLINEAR(RANK(CORR((LOW),MEAN(VOLUME,80), 8)), 17))) * -1)

In [69]:
Alpha62 = (-1 * CORR(HIGH, RANK(VOLUME), 5))

In [70]:
Alpha63 = SMA(MAX(CLOSE-DELAY(CLOSE,1),0),6,1)/SMA(ABS(CLOSE-DELAY(CLOSE,1)),6,1)*100

In [71]:
Alpha64 = (MAX(RANK(DECAYLINEAR(CORR(RANK(VWAP), RANK(VOLUME), 4),4)),RANK(DECAYLINEAR(MAX(CORR(RANK(CLOSE), RANK(MEAN(VOLUME,60)), 4), 13), 14))) * -1)

In [72]:
Alpha65 = MEAN(CLOSE,6)/CLOSE

In [73]:
Alpha66 = (CLOSE-MEAN(CLOSE,6))/MEAN(CLOSE,6)*100

In [74]:
Alpha67 = (CLOSE-MEAN(CLOSE,6))/MEAN(CLOSE,6)*100

In [75]:
Alpha68 = SMA(((HIGH+LOW)/2-(DELAY(HIGH,1)+DELAY(LOW,1))/2)*(HIGH-LOW)/VOLUME,15,2)

In [76]:
Alpha69 = (SUM(DTM,20)>SUM(DBM,20)) * (SUM(DTM,20)-SUM(DBM,20))/SUM(DTM,20) + \
        ~(SUM(DTM,20)>SUM(DBM,20))*((SUM(DTM,20)==SUM(DBM,20))*0 + (SUM(DTM,20)!=SUM(DBM,20))*(SUM(DTM,20)-SUM(DBM,20))/SUM(DBM,20))

In [77]:
Alpha70 = STD(AMOUNT,6)

In [78]:
Alpha71 = (CLOSE-MEAN(CLOSE,24))/MEAN(CLOSE,24)*100

In [79]:
Alpha72 = SMA((TSMAX(HIGH,6)-CLOSE)/(TSMAX(HIGH,6)-TSMIN(LOW,6))*100,15,1)

In [80]:
Alpha73 = ((TSRANK(DECAYLINEAR(DECAYLINEAR(CORR((CLOSE), VOLUME,10),16),4),5) -RANK(DECAYLINEAR(CORR(VWAP, MEAN(VOLUME,30), 4),3))) * -1)

In [81]:
Alpha74 = (RANK(CORR(SUM(((LOW * 0.35) + (VWAP*0.65)), 20), SUM(MEAN(VOLUME,40),20), 7)) +RANK(CORR(RANK(VWAP), RANK(VOLUME), 6))) 

In [82]:
#Alpha75 drop, 因为需要用到那个什么BENCHMARK

In [83]:
Alpha76 = STD(ABS((CLOSE/DELAY(CLOSE,1)-1))/VOLUME,20)/MEAN(ABS((CLOSE/DELAY(CLOSE,1)-1))/VOLUME,20)

In [84]:
Alpha77 = MIN(RANK(DECAYLINEAR(((((HIGH + LOW) / 2) + HIGH) - (VWAP+HIGH)),20)),RANK(DECAYLINEAR(CORR(((HIGH + LOW) / 2), MEAN(VOLUME,40), 3), 6)))

In [85]:
"这里将MA()替换为MEAN()"
Alpha78 = ((HIGH+LOW+CLOSE)/3-MEAN((HIGH+LOW+CLOSE)/3,12))/(0.015*MEAN(ABS(CLOSE-MEAN((HIGH+LOW+CLOSE)/3,12)),12))

In [86]:
Alpha79 = SMA(MAX(CLOSE-DELAY(CLOSE,1),0),12,1)/SMA(ABS(CLOSE-DELAY(CLOSE,1)),12,1)*100

In [87]:
Alpha80 = (VOLUME-DELAY(VOLUME,5))/DELAY(VOLUME,5)*100

In [88]:
Alpha81 = SMA(VOLUME,21,2)

In [89]:
Alpha82 = SMA((TSMAX(HIGH,6)-CLOSE)/(TSMAX(HIGH,6)-TSMIN(LOW,6))*100,20,1)

In [90]:
Alpha83 = (-1 * RANK(COVIANCE(RANK(HIGH), RANK(VOLUME), 5)))

In [91]:
Alpha84 = SUM((CLOSE>DELAY(CLOSE,1)*VOLUME+ \
               ~(CLOSE>DELAY(CLOSE,1))*(-VOLUME)*(CLOSE<DELAY(CLOSE,1))),20)

In [92]:
Alpha85 = (TSRANK((VOLUME / MEAN(VOLUME,20)), 20) * TSRANK((-1 * DELTA(CLOSE, 7)), 8))

In [93]:
Alpha86 = (0.25 < (((DELAY(CLOSE, 20) - DELAY(CLOSE, 10)) / 10) - ((DELAY(CLOSE, 10) - CLOSE) / 10))) *(-1)\
+ ~(0.25 < (((DELAY(CLOSE, 20) - DELAY(CLOSE, 10)) / 10) - ((DELAY(CLOSE, 10) - CLOSE) / 10)))\
* (((((DELAY(CLOSE, 20) - DELAY(CLOSE, 10)) / 10) - ((DELAY(CLOSE, 10) - CLOSE) / 10)) < 0) * 1\
    + ~((((DELAY(CLOSE, 20) - DELAY(CLOSE, 10)) / 10) - ((DELAY(CLOSE, 10) - CLOSE) / 10)) < 0)\
    *((-1 * 1) * (CLOSE - DELAY(CLOSE, 1))))

In [94]:
Alpha87 = ((RANK(DECAYLINEAR(DELTA(VWAP, 4), 7)) + TSRANK(DECAYLINEAR(((((LOW * 0.9) + (LOW * 0.1)) - VWAP) / (OPEN - ((HIGH + LOW) / 2))), 11), 7)) * -1)

In [95]:
Alpha88 = (CLOSE-DELAY(CLOSE,20))/DELAY(CLOSE,20)*100

In [96]:
Alpha89 = 2*(SMA(CLOSE,13,2)-SMA(CLOSE,27,2)-SMA(SMA(CLOSE,13,2)-SMA(CLOSE,27,2),10,2))

In [97]:
Alpha90 = ( RANK(CORR(RANK(VWAP), RANK(VOLUME), 5)) * -1)

In [98]:
Alpha91 = ((RANK((CLOSE - MAX(CLOSE, 5)))*RANK(CORR((MEAN(VOLUME,40)), LOW, 5))) * -1)

In [99]:
Alpha92 = MAX(RANK(DECAYLINEAR(DELTA(((CLOSE* 0.35)+ (VWAP*0.65)),2),3)),\
              TSRANK(DECAYLINEAR(ABS(CORR((MEAN(VOLUME,180)), CLOSE, 13)), 5), 15)) * -1

In [100]:
Alpha93 = SUM((OPEN>=DELAY(OPEN,1)*0+ ~(OPEN>=DELAY(OPEN,1))*MAX((OPEN-LOW),(OPEN-DELAY(OPEN,1)))),20)

In [101]:
Alpha94 = SUM((CLOSE>DELAY(CLOSE,1)*VOLUME+ \
               ~(CLOSE>DELAY(CLOSE,1))*(-VOLUME)*(CLOSE<DELAY(CLOSE,1))),30)

In [102]:
Alpha95 = STD(AMOUNT,20)

In [103]:
Alpha96 = SMA(SMA((CLOSE-TSMIN(LOW,9))/(TSMAX(HIGH,9)-TSMIN(LOW,9))*100,3,1),3,1)

In [104]:
Alpha97 = STD(VOLUME,10)

In [105]:
condition = (((DELTA((SUM(CLOSE, 100) / 100), 100) / DELAY(CLOSE, 100)) < 0.05) | ((DELTA((SUM(CLOSE, 100) / 100), 100) / DELAY(CLOSE, 100)) == 0.05))

Alpha98 = ( condition * (-1 * (CLOSE - TSMIN(CLOSE, 100))) + ~condition * (-1 * DELTA(CLOSE, 3)))

In [106]:
Alpha99 = (-1 * RANK(COVIANCE(RANK(CLOSE), RANK(VOLUME), 5)))

In [107]:
Alpha100 = STD(VOLUME,20)

In [108]:
Alpha101 = ((RANK(CORR(CLOSE, SUM(MEAN(VOLUME,30), 37), 15)) < RANK(CORR(RANK(((HIGH * 0.1) + (VWAP * 0.9))), RANK(VOLUME), 11))) * -1)

In [109]:
Alpha102 = SMA(MAX(VOLUME-DELAY(VOLUME,1),0),6,1)/SMA(ABS(VOLUME-DELAY(VOLUME,1)),6,1)*100

In [110]:
Alpha103 = ((20-LOWDAY(LOW,20))/20)*100

In [111]:
Alpha104 = (-1 * (DELTA(CORR(HIGH, VOLUME, 5), 5) * RANK(STD(CLOSE, 20))))

In [112]:
Alpha105 = (-1 * CORR(RANK(OPEN), RANK(VOLUME), 10))

In [113]:
Alpha106 = CLOSE-DELAY(CLOSE,20)

In [114]:
Alpha107 = (((-1 * RANK((OPEN - DELAY(HIGH, 1)))) * RANK((OPEN - DELAY(CLOSE, 1)))) * RANK((OPEN - DELAY(LOW, 1))))

In [115]:
Alpha108 = ((RANK((HIGH - MIN(HIGH, 2)))**RANK(CORR((VWAP), (MEAN(VOLUME,120)), 6))) * -1)

In [116]:
Alpha109 = SMA(HIGH-LOW,10,2)/SMA(SMA(HIGH-LOW,10,2),10,2)

In [117]:
Alpha110 = SUM(MAX(0,HIGH-DELAY(CLOSE,1)),20)/SUM(MAX(0,DELAY(CLOSE,1)-LOW),20)*100

In [118]:
Alpha111 = SMA(VOLUME*((CLOSE-LOW)-(HIGH-CLOSE))/(HIGH-LOW),11,2)\
    -SMA(VOLUME*((CLOSE-LOW)-(HIGH-CLOSE))/(HIGH-LOW),4,2)

In [119]:
#这个也是A/B*C的形式
Alpha112 = (SUM((CLOSE-DELAY(CLOSE,1)>0) * (CLOSE-DELAY(CLOSE,1)),12)\
 -SUM(((CLOSE-DELAY(CLOSE,1)<0) * ABS(CLOSE-DELAY(CLOSE,1))),12))\
/(SUM((CLOSE-DELAY(CLOSE,1)>0) * (CLOSE-DELAY(CLOSE,1)),12)\
  +SUM(((CLOSE-DELAY(CLOSE,1)<0) * ABS(CLOSE-DELAY(CLOSE,1))),12))*100

In [120]:
Alpha113 = (-1 * ((RANK((SUM(DELAY(CLOSE, 5), 20) / 20)) * CORR(CLOSE, VOLUME, 2)) * RANK(CORR(SUM(CLOSE, 5), SUM(CLOSE, 20), 2))))

In [121]:
Alpha114 = ((RANK(DELAY(((HIGH - LOW) / (SUM(CLOSE, 5) / 5)), 2)) * RANK(RANK(VOLUME))) / (((HIGH - LOW) / (SUM(CLOSE, 5) / 5)) / (VWAP - CLOSE)))

In [122]:
Alpha115 = RANK(CORR(((HIGH * 0.9) + (CLOSE * 0.1)), MEAN(VOLUME,30), 10))**RANK(CORR(TSRANK(((HIGH + LOW) / 2), 4), TSRANK(VOLUME, 10), 7))

In [123]:
Alpha116 = REGBETA(CLOSE,SEQUENCE(20),20)

In [124]:
Alpha117 = ((TSRANK(VOLUME, 32) * (1 - TSRANK(((CLOSE + HIGH) - LOW), 16))) * (1 - TSRANK(RET, 32)))

In [125]:
Alpha118 = SUM(HIGH-OPEN,20)/SUM(OPEN-LOW,20)*100

In [126]:
Alpha119 = (RANK(DECAYLINEAR(CORR(VWAP, SUM(MEAN(VOLUME,5), 26), 5),7))-RANK(DECAYLINEAR(TSRANK(MIN(CORR(RANK(OPEN), RANK(MEAN(VOLUME,15)), 21), 9), 7), 8)))

In [127]:
Alpha120 = (RANK((VWAP - CLOSE)) / RANK((VWAP + CLOSE)))

In [128]:
Alpha121 = ((RANK((VWAP - MIN(VWAP, 12)))**TSRANK(CORR(TSRANK(VWAP, 20), TSRANK(MEAN(VOLUME,60), 2), 18), 3)) * -1)

In [129]:
Alpha122 = (SMA(SMA(SMA(np.log(CLOSE),13,2),13,2),13,2)-DELAY(SMA(SMA(SMA(np.log(CLOSE),13,2),13,2),13,2),1))/DELAY(SMA(SMA(SMA(np.log(CLOSE),13,2),13,2),13,2),1)

In [130]:
Alpha123 = ((RANK(CORR(SUM(((HIGH + LOW) / 2), 20), SUM(MEAN(VOLUME,60), 20), 9)) < RANK(CORR(LOW, VOLUME, 6))) * -1)

In [131]:
Alpha124 = (CLOSE - VWAP) / DECAYLINEAR(RANK(TSMAX(CLOSE, 30)),2)

In [132]:
Alpha125 = (RANK(DECAYLINEAR(CORR((VWAP), MEAN(VOLUME,80),17), 20)) / RANK(DECAYLINEAR(DELTA(((CLOSE * 0.5) + (VWAP * 0.5)), 3), 16)))

In [133]:
Alpha126 = (CLOSE+HIGH+LOW)/3

In [134]:
Alpha127 = (MEAN((100*(CLOSE-MAX(CLOSE,12))/(MAX(CLOSE,12))), 12)**2)**(1/2)

In [135]:
Alpha128 = 100-(100/(1+SUM(((HIGH+LOW+CLOSE)/3>DELAY((HIGH+LOW+CLOSE)/3,1))*(HIGH+LOW+CLOSE)/3*VOLUME,14)/
                     SUM(((HIGH+LOW+CLOSE)/3<DELAY((HIGH+LOW+CLOSE)/3,1)) * (HIGH+LOW+CLOSE)/3*VOLUME, 14)))

In [136]:
Alpha129 = SUM((CLOSE-DELAY(CLOSE,1)<0) * ABS(CLOSE-DELAY(CLOSE,1)),12)

In [137]:
Alpha130 = (RANK(DECAYLINEAR(CORR(((HIGH + LOW) / 2),MEAN(VOLUME,40),9),10))/RANK(DECAYLINEAR(CORR(RANK(VWAP), RANK(VOLUME), 7),3)))

In [138]:
Alpha131 = (RANK(DELAY(VWAP, 1))**TSRANK(CORR(CLOSE,MEAN(VOLUME,50), 18), 18))

In [139]:
Alpha132 = MEAN(AMOUNT,20)

In [140]:
Alpha133 = ((20-HIGHDAY(HIGH,20))/20)*100-((20-LOWDAY(LOW,20))/20)*100

In [141]:
Alpha134 = (CLOSE-DELAY(CLOSE,12))/DELAY(CLOSE,12)*VOLUME

In [142]:
Alpha135 = SMA(DELAY(CLOSE/DELAY(CLOSE,20),1),20,1)

In [143]:
Alpha136 = ((-1 * RANK(DELTA(RET, 3))) * CORR(OPEN, VOLUME, 10))

In [144]:
#这个就是Alpha55把最外面那层sum()去掉
part_C = MAX(ABS(HIGH-DELAY(CLOSE,1)),ABS(LOW-DELAY(CLOSE,1)))

part_A = (CLOSE-DELAY(CLOSE,1)+(CLOSE-OPEN)/2+DELAY(CLOSE,1)-DELAY(OPEN,1))

part_B1_cond = (ABS(HIGH-DELAY(CLOSE,1))>ABS(LOW-DELAY(CLOSE,1))) & \
    (ABS(HIGH-DELAY(CLOSE,1))>ABS(HIGH-DELAY(LOW,1)))
    
part_B2_cond = (ABS(LOW-DELAY(CLOSE,1))>ABS(HIGH-DELAY(LOW,1))) & \
    (ABS(LOW-DELAY(CLOSE,1))>ABS(HIGH-DELAY(CLOSE,1)))

part_B = part_B1_cond *(ABS(HIGH-DELAY(CLOSE,1))+ABS(LOW-DELAY(CLOSE,1))/2 + ABS(DELAY(CLOSE,1)-DELAY(OPEN,1))/4)\
            + (~part_B1_cond)\
            * (part_B2_cond * (ABS(LOW-DELAY(CLOSE,1))+ABS(HIGH-DELAY(CLOSE,1))/2 + ABS(DELAY(CLOSE,1)-DELAY(OPEN,1))/4)\
            + (~part_B2_cond) * (ABS(HIGH-DELAY(LOW,1))+ABS(DELAY(CLOSE,1)-DELAY(OPEN,1))/4))

Alpha137 = part_A/part_B*part_C

In [145]:
Alpha138 = ((RANK(DECAYLINEAR(DELTA((((LOW * 0.7) + (VWAP *0.3))), 3), 20))-TSRANK(DECAYLINEAR(TSRANK(CORR(TSRANK(LOW, 8), TSRANK(MEAN(VOLUME,60), 17), 5), 19), 16), 7)) * -1)

In [146]:
Alpha139 = (-1 * CORR(OPEN, VOLUME, 10))

In [147]:
Alpha140 = MIN(RANK(DECAYLINEAR(((RANK(OPEN) + RANK(LOW)) - (RANK(HIGH) + RANK(CLOSE))),8)),TSRANK(DECAYLINEAR(CORR(TSRANK(CLOSE, 8), TSRANK(MEAN(VOLUME,60), 20), 8), 7), 3))

In [148]:
Alpha141 = (RANK(CORR(RANK(HIGH), RANK(MEAN(VOLUME,15)), 9))* -1)

In [149]:
Alpha142 = (((-1 * RANK(TSRANK(CLOSE, 10))) * RANK(DELTA(DELTA(CLOSE,1), 1))) *RANK(TSRANK((VOLUME/MEAN(VOLUME,20)), 5)))

In [150]:
#Alpha143,没有定义SELF函数

In [151]:
Alpha144 = SUMIF(ABS(CLOSE/DELAY(CLOSE,1)-1)/AMOUNT,20,CLOSE<DELAY(CLOSE,1))\
            /COUNT(CLOSE<DELAY(CLOSE, 1),20)

In [152]:
Alpha145 = (MEAN(VOLUME,9)-MEAN(VOLUME,26))/MEAN(VOLUME,12)*100

In [153]:
Alpha146 = MEAN((CLOSE-DELAY(CLOSE,1))/DELAY(CLOSE,1)-SMA((CLOSE-DELAY(CLOSE,1))/DELAY(CLOSE,1),61,2),20)\
*(( CLOSE-DELAY(CLOSE,1))/DELAY(CLOSE,1)-SMA((CLOSE-DELAY(CLOSE,1))/DELAY(CLOSE,1),61,2))/SMA(((CLOSE-DELAY(CLOSE,1))\
/DELAY(CLOSE,1)-((CLOSE-DELAY(CLOSE,1))/DELAY(CLOSE,1)-SMA((CLOSE-DELAY(CLOSE, 1))/DELAY(CLOSE,1),61,2)))**2,61,2);

In [154]:
Alpha147 = REGBETA(MEAN(CLOSE,12), SEQUENCE(12), 12)

In [155]:
Alpha148 = ((RANK(CORR((OPEN), SUM(MEAN(VOLUME,60), 9), 6)) < RANK((OPEN - TSMIN(OPEN, 14)))) * -1)

In [156]:
#Alpha149, BANCHMARKCLOSE没有定义，所以这个index空着

In [157]:
Alpha150 = (CLOSE+HIGH+LOW)/3*VOLUME

In [158]:
Alpha151 = SMA(CLOSE-DELAY(CLOSE,20),20,1)

In [159]:
Alpha152 = SMA(MEAN(DELAY(SMA(DELAY(CLOSE/DELAY(CLOSE,9),1),9,1),1),12)-MEAN(DELAY(SMA(DELAY(CLOSE/DELAY (CLOSE,9),1),9,1),1),26),9,1)

In [160]:
Alpha153 = (MEAN(CLOSE,3)+MEAN(CLOSE,6)+MEAN(CLOSE,12)+MEAN(CLOSE,24))/4

In [161]:
Alpha154 = (((VWAP - MIN(VWAP, 16))) < (CORR(VWAP, MEAN(VOLUME,180), 18)))

In [162]:
Alpha155 = SMA(VOLUME,13,2)-SMA(VOLUME,27,2)-SMA(SMA(VOLUME,13,2)-SMA(VOLUME,27,2),10,2)

In [163]:
Alpha156 = (MAX(RANK(DECAYLINEAR(DELTA(VWAP, 5), 3)), RANK(DECAYLINEAR(((DELTA(((OPEN * 0.15) + (LOW *0.85)), 2) / ((OPEN * 0.15) + (LOW * 0.85))) * -1), 3))) * -1)

In [164]:
Alpha157 = (MIN(PROD(RANK(RANK(LOG(SUM(TSMIN(RANK(RANK((-1 * RANK(DELTA((CLOSE - 1), 5))))), 2), 1)))), 1), 5) + TSRANK(DELAY((-1 * RET), 6), 5))

In [165]:
Alpha158 = ((HIGH-SMA(CLOSE,15,2))-(LOW-SMA(CLOSE,15,2)))/CLOSE

In [166]:
Alpha159 = ((CLOSE-SUM(MIN(LOW,DELAY(CLOSE,1)),6))/SUM(MAX(HIGH,DELAY(CLOSE,1))-MIN(LOW,DELAY(CLOSE,1)),6) *12*24+(CLOSE-SUM(MIN(LOW,DELAY(CLOSE,1)),12))/SUM(MAX(HIGH,DELAY(CLOSE,1))-MIN(LOW,DELAY(CLOSE,1)),12)*6*24+(CLOSE-SUM(MIN(LOW,DELAY(CLOSE,1)),24))/SUM(MAX(HIGH,DELAY(CLOSE,1))-MIN(LOW,DELAY(CLOSE,1)),24)*6*24)*100/(6*12+6*24+12*24)

In [167]:
Alpha160 = SMA((CLOSE<=DELAY(CLOSE,1))*STD(CLOSE,20),20,1)

In [168]:
Alpha161 = MEAN(MAX(MAX((HIGH-LOW),ABS(DELAY(CLOSE,1)-HIGH)),ABS(DELAY(CLOSE,1)-LOW)),12)

In [169]:
Alpha162 = (SMA(MAX(CLOSE-DELAY(CLOSE,1),0),12,1)/SMA(ABS(CLOSE-DELAY(CLOSE,1)),12,1)*100-MIN(SMA(MAX(CLOSE-DELAY(CLOSE,1),0),12,1)/SMA(ABS(CLOSE-DELAY(CLOSE,1)),12,1)*100,12))/(MAX(SMA(MAX(CLOSE-DELAY(CLOSE,1),0),12,1)/SMA(ABS(CLOSE-DELAY(CLOSE,1)),12,1)*100,12)-MIN(SMA(MAX(CLOSE-DELAY(CLOSE,1),0),12, 1)/SMA(ABS(CLOSE-DELAY(CLOSE,1)),12,1)*100,12))

In [170]:
Alpha163 = RANK(((((-1 * RET) * MEAN(VOLUME,20)) * VWAP) * (HIGH - CLOSE)))

In [171]:
Alpha164 = SMA((((CLOSE>DELAY(CLOSE,1))*1/(CLOSE-DELAY(CLOSE,1))+ ~(CLOSE>DELAY(CLOSE,1))*1)
        - MIN(((CLOSE>DELAY(CLOSE,1))*1/(CLOSE-DELAY(CLOSE,1))+ ~(CLOSE>DELAY(CLOSE,1))*1),12))/(HIGH-LOW)*100,13,2)

In [172]:
'''SUMAC(A,n)没有给窗口n，故此设为48，但是max和min好像也只传入了一个series，所以我把max和min都删掉了。。。'''
Alpha165 = SUMAC(CLOSE-MEAN(CLOSE,48),48)- SUMAC(CLOSE-MEAN(CLOSE,48),48)/STD(CLOSE,48)

In [173]:
Alpha166 = -20*((20-1)**1.5)*SUM((CLOSE/DELAY(CLOSE,1)-MEAN(CLOSE/DELAY(CLOSE,1)-1,20)),20) \
            /((20-1)*(20-2)*((SUM((CLOSE/DELAY(CLOSE,1))**2,20))**1.5))

In [174]:
Alpha167 = SUM((CLOSE-DELAY(CLOSE,1)>0)*CLOSE-DELAY(CLOSE,1),12)

In [175]:
Alpha168 = (-1*VOLUME/MEAN(VOLUME,20))

In [176]:
Alpha169 = SMA(MEAN(DELAY(SMA(CLOSE-DELAY(CLOSE,1),9,1),1),12)-MEAN(DELAY(SMA(CLOSE-DELAY(CLOSE,1),9,1),1), 26),10,1)

In [177]:
Alpha170 = ((((RANK((1 / CLOSE)) * VOLUME) / MEAN(VOLUME,20)) * ((HIGH * RANK((HIGH - CLOSE))) / (SUM(HIGH, 5) / 5))) - RANK((VWAP - DELAY(VWAP, 5))))

In [178]:
Alpha171 = ((-1 * ((LOW - CLOSE) * (OPEN**5))) / ((CLOSE - HIGH) * (CLOSE**5)))

In [179]:
Alpha172 = SMA((CLOSE>DELAY(CLOSE,1))*STD(CLOSE,20),20,1)

In [180]:
Alpha173 = 3*SMA(CLOSE,13,2)-2*SMA(SMA(CLOSE,13,2),13,2)+SMA(SMA(SMA(np.log(CLOSE),13,2),13,2),13,2)

In [181]:
Alpha174 = SMA((CLOSE>DELAY(CLOSE,1))*STD(CLOSE,20),20,1)

In [182]:
Alpha175 = MEAN(MAX(MAX((HIGH-LOW),ABS(DELAY(CLOSE,1)-HIGH)),ABS(DELAY(CLOSE,1)-LOW)),6)

In [183]:
Alpha176 = CORR(RANK(((CLOSE - TSMIN(LOW, 12)) / (TSMAX(HIGH, 12) - TSMIN(LOW,12)))), RANK(VOLUME), 6)

In [184]:
Alpha177 = ((20-HIGHDAY(HIGH,20))/20)*100

In [185]:
Alpha178 = (CLOSE-DELAY(CLOSE,1))/DELAY(CLOSE,1)*VOLUME

In [186]:
Alpha179 = (RANK(CORR(VWAP, VOLUME, 4)) *RANK(CORR(RANK(LOW), RANK(MEAN(VOLUME,50)), 12)))

In [187]:
Alpha180 = ((MEAN(VOLUME,20) < VOLUME) * ((-1 * TSRANK(ABS(DELTA(CLOSE, 7)), 60)) * SIGN(DELTA(CLOSE, 7))\
            + ~(MEAN(VOLUME,20) < VOLUME) * (-1 * VOLUME)))

In [188]:
#Alpha181 drop for the BENCHMARKINDEX

In [189]:
#Alpha182 drop for the BENCHMARKINDEX

In [190]:
Alpha183 = MAX(SUMAC(CLOSE-MEAN(CLOSE,24),24),24)-MIN(SUMAC(CLOSE-MEAN(CLOSE,24),24),24)/STD(CLOSE,24)

In [191]:
Alpha184 = (RANK(CORR(DELAY((OPEN - CLOSE), 1), CLOSE, 200)) + RANK((OPEN - CLOSE)))

In [192]:
Alpha185 = RANK((-1 * ((1 - (OPEN / CLOSE))**2)))

In [193]:
Alpha186 = (MEAN(ABS(SUM(((LD>0) & (LD>HD))*LD,14)*100/SUM(TR,14)\
                    -SUM(((HD>0) & (HD>LD))*HD,14)*100/SUM(TR,14))\
                   /(SUM(((LD>0) & (LD>HD))*LD,14)*100/SUM(TR,14)\
                    +SUM(((HD>0) & (HD>LD))*HD,14)*100/SUM(TR,14))*100,6)\
     +DELAY(MEAN(ABS(SUM(((LD>0) & (LD>HD))*LD,14)*100/SUM(TR,14)\
                    -SUM(((HD>0) & (HD>LD))*HD,14)*100/SUM(TR,14))\
                   /(SUM(((LD>0) & (LD>HD))*LD,14)*100/SUM(TR,14)\
                    +SUM(((HD>0) & (HD>LD))*HD,14)*100/SUM(TR,14))*100,6),6))/2


In [194]:
Alpha187 = SUM(~(OPEN<=DELAY(OPEN,1)) * MAX((HIGH-OPEN),(OPEN-DELAY(OPEN,1))),20)

In [195]:
Alpha188 = ((HIGH-LOW-SMA(HIGH-LOW,11,2))/SMA(HIGH-LOW,11,2))*100

In [196]:
Alpha189 = MEAN(ABS(CLOSE-MEAN(CLOSE,6)),6)

In [197]:
Alpha190 = np.log((COUNT((CLOSE/DELAY(CLOSE)-1)>((CLOSE/DELAY(CLOSE,19))**(1/20)-1),20)-1)\
        *SUMIF(((CLOSE/DELAY(CLOSE,1)-(CLOSE/DELAY(CLOSE,19))**(1/20)-1))**2,20,\
                CLOSE/DELAY(CLOSE,1)<((CLOSE/DELAY(CLOSE,19))**(1/20)-1))\
                  
        /(
            COUNT(CLOSE/DELAY(CLOSE,1)<((CLOSE/DELAY(CLOSE,19))**(1/20)-1),20)\
        *SUMIF(CLOSE/DELAY(CLOSE,1)-((CLOSE/DELAY(CLOSE,19))**(1/20)-1)**2,20,\
                CLOSE/DELAY(CLOSE,1)>((CLOSE/DELAY(CLOSE,19))**(1/20)-1))\
        ))

In [198]:
Alpha191 = ((CORR(MEAN(VOLUME,20), LOW, 5) + ((HIGH + LOW) / 2)) - CLOSE)

In [199]:
process_inf(Alpha1)
#process_inf(Alpha1) - Alpha1 

DataFrame has infinity values


2018-02-01 00:00:10    1.000000
2018-02-01 00:00:20    1.000000
2018-02-01 00:00:30    0.986811
2018-02-01 00:00:40    0.990591
2018-02-01 00:02:40   -0.074268
2018-02-01 00:02:50   -0.161386
2018-02-01 00:04:30    0.060422
2018-02-01 00:04:40   -0.007739
2018-02-01 00:04:50   -0.012673
2018-02-01 00:05:00    0.237409
2018-02-01 00:05:10   -0.678406
2018-02-01 00:05:20   -0.799450
2018-02-01 00:07:20   -0.325109
2018-02-01 00:07:30   -0.390568
2018-02-01 00:07:40   -0.280476
2018-02-01 00:08:50   -0.451181
2018-02-01 00:09:00   -0.468995
2018-02-01 00:09:10    0.889148
2018-02-01 00:09:20    0.719530
2018-02-01 00:09:30   -0.011630
2018-02-01 00:09:40    0.376439
2018-02-01 00:09:50    0.192933
2018-02-01 00:10:00   -0.203718
2018-02-01 00:11:50   -0.050300
2018-02-01 00:12:00    0.046174
2018-02-01 00:12:10   -0.096664
2018-02-01 00:12:20   -0.168548
2018-02-01 00:13:50   -0.075210
2018-02-01 00:14:00   -0.001054
2018-02-01 00:14:10    0.434775
                         ...   
2018-04-

In [200]:
判断信号好不好的方法是，看未来五分钟的收益率，作为target
用pearson correlation来看每个信号跟收益率相关性有多少

设置网格搜索 寻找最优参数

设计一个 搜索最优

Alpha-index线性组合，求他前面最大的参数

搜索，如果搜到了就把新的字符串返回，判断是不是数字，判断这个alpha里面有多少个变量是可以需要修改的


eval() 函数，可以自动执行这段字符串 然后得到他的结果

SyntaxError: invalid character in identifier (<ipython-input-200-01c61de855a6>, line 1)